In [0]:
from pyspark.sql.functions import current_timestamp
def add_ingestion_date(input_df):
  output_df = input_df.withColumn("ingestion_date", current_timestamp())
  return output_df

In [0]:
def ovewrite_partition(input_df,db_name,table_name,column_partition):
    for item_list in input_df.select(f"{column_partition}").distinct().collect():
        if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):#Solo se ejecuta el spark sql si existe la tabla, es una validacion
            spark.sql(f"ALTER TABLE {db_name}.{table_name} DROP IF EXISTS PARTITION({column_partition}='{item_list[column_partition]}')")#Con collect cargo los datos en memoria| Con el comando SQL borro la particion existente asi evito duplicados.

In [0]:
from delta.tables import *

def merge_delta_lake(input_df,db_name,table_name,folder_path,merge_condition,column_partition):    

    if(spark._jsparkSession.catalog().tableExists(f"{db_name}.{table_name}")):
    
        deltaTable = DeltaTable.forPath(spark, f'{folder_path}/{table_name}/')

        deltaTable.alias('tgt')\
            .merge(
            input_df.alias('src'),
            merge_condition
            ) \
            .whenMatchedUpdateAll()\
            .whenNotMatchedInsertAll()\
            .execute()
        
    else:
        input_df.write.mode("overwrite").partitionBy(column_partition).format("delta").saveAsTable(f"{db_name}.{table_name}")#Con mode("overwrite") sobreescribo el archivo y me evito errores| Con partitionBy particiono el archivo para guardarse por el campo que indique